Dataset recreation
===================

Here we are going to explore the SBB Google Drive dataset with the last two years of data by taking a random day. We are going to clean it since it also contains information about busses and other railway networks. For graph plotting, we will add the geolocations of the stations, loaded from https://opendata.swiss/de/dataset/dienststellen-gemass-opentransportdata-swiss. We then automate this process by using a python script for all the other days.

## Imports

In [1]:
%matplotlib inline
import pandas as pd
print('pandas: {}'.format(pd.__version__))
import numpy as np
print('numpy: {}'.format(np.__version__))
import seaborn as sns
print('seaborn: {}'.format(sns.__version__))
import geopy as geo
print('geopy: {}'.format(geo.__version__))
import datetime

import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import powerlaw
from geopy import distance

pandas: 0.25.1
numpy: 1.17.2
seaborn: 0.9.0
geopy: 1.20.0


## Dataset recreation

In [12]:
# Reference file
filepath = '2018-01-01istdaten.csv'
df = pd.read_csv(filepath, delimiter=';')
df.columns

/home/mapa/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['BETRIEBSTAG', 'FAHRT_BEZEICHNER', 'BETREIBER_ID', 'BETREIBER_ABK',
       'BETREIBER_NAME', 'PRODUKT_ID', 'LINIEN_ID', 'LINIEN_TEXT', 'UMLAUF_ID',
       'VERKEHRSMITTEL_TEXT', 'ZUSATZFAHRT_TF', 'FAELLT_AUS_TF', 'BPUIC',
       'HALTESTELLEN_NAME', 'ANKUNFTSZEIT', 'AN_PROGNOSE',
       'AN_PROGNOSE_STATUS', 'ABFAHRTSZEIT', 'AB_PROGNOSE',
       'AB_PROGNOSE_STATUS', 'DURCHFAHRT_TF'],
      dtype='object')

In [13]:
df.shape

(596809, 21)

In [14]:
# Drop anything that is no train ('Zug')
df = df.drop(df[(df['PRODUKT_ID'] != 'Zug')].index)
df = df.drop(df[(df['BETREIBER_ABK'] != 'SBB')].index)
df.head()

,BETRIEBSTAG,FAHRT_BEZEICHNER,BETREIBER_ID,BETREIBER_ABK,BETREIBER_NAME,PRODUKT_ID,LINIEN_ID,LINIEN_TEXT,UMLAUF_ID,VERKEHRSMITTEL_TEXT,...,FAELLT_AUS_TF,BPUIC,HALTESTELLEN_NAME,ANKUNFTSZEIT,AN_PROGNOSE,AN_PROGNOSE_STATUS,ABFAHRTSZEIT,AB_PROGNOSE,AB_PROGNOSE_STATUS,DURCHFAHRT_TF
212,01.01.2018,80:L7____:87875:000,80:L7____,SBB,SBB GmbH,Zug,87875,S6,NaN,S,...,False,8500090,Basel Bad Bf,01.01.2018 01:14,NaN,UNBEKANNT,NaN,NaN,PROGNOSE,False
314,01.01.2018,85:11:10:002,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,10,EC,NaN,EC,...,False,8505307,Chiasso,01.01.2018 19:10,01.01.2018 19:11:15,PROGNOSE,01.01.2018 19:15,01.01.2018 19:17:01,PROGNOSE,False
315,01.01.2018,85:11:10:002,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,10,EC,NaN,EC,...,False,8505300,Lugano,01.01.2018 19:40,01.01.2018 19:38:14,GESCHAETZT,01.01.2018 19:42,01.01.2018 19:43:14,GESCHAETZT,False
316,01.01.2018,85:11:10:002,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,10,EC,NaN,EC,...,False,8505213,Bellinzona,01.01.2018 20:10,01.01.2018 20:08:39,GESCHAETZT,01.01.2018 20:13,01.01.2018 20:14:36,GESCHAETZT,False
317,01.01.2018,85:11:10:002,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,10,EC,NaN,EC,...,False,8505004,Arth-Goldau,01.01.2018 21:09,01.01.2018 21:08:24,GESCHAETZT,01.01.2018 21:13,01.01.2018 21:14:10,GESCHAETZT,False


In [15]:
df.shape

(55167, 21)

Columns **not** present: ankunftsverspatung, abfahrtsverspatung, lod, geopos, GdeNummer

### 1. Train's arrival is delayed (ankunftsverspatung)


In [16]:
# Train's arrival is delayed
df.loc[(df['ANKUNFTSZEIT'].notnull()) & (df['AN_PROGNOSE'].notnull()), 'ankunftsverspatung'] = (df['ANKUNFTSZEIT'] < df['AN_PROGNOSE'])
df.head()

,BETRIEBSTAG,FAHRT_BEZEICHNER,BETREIBER_ID,BETREIBER_ABK,BETREIBER_NAME,PRODUKT_ID,LINIEN_ID,LINIEN_TEXT,UMLAUF_ID,VERKEHRSMITTEL_TEXT,...,BPUIC,HALTESTELLEN_NAME,ANKUNFTSZEIT,AN_PROGNOSE,AN_PROGNOSE_STATUS,ABFAHRTSZEIT,AB_PROGNOSE,AB_PROGNOSE_STATUS,DURCHFAHRT_TF,ankunftsverspatung
212,01.01.2018,80:L7____:87875:000,80:L7____,SBB,SBB GmbH,Zug,87875,S6,NaN,S,...,8500090,Basel Bad Bf,01.01.2018 01:14,NaN,UNBEKANNT,NaN,NaN,PROGNOSE,False,NaN
314,01.01.2018,85:11:10:002,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,10,EC,NaN,EC,...,8505307,Chiasso,01.01.2018 19:10,01.01.2018 19:11:15,PROGNOSE,01.01.2018 19:15,01.01.2018 19:17:01,PROGNOSE,False,True
315,01.01.2018,85:11:10:002,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,10,EC,NaN,EC,...,8505300,Lugano,01.01.2018 19:40,01.01.2018 19:38:14,GESCHAETZT,01.01.2018 19:42,01.01.2018 19:43:14,GESCHAETZT,False,False
316,01.01.2018,85:11:10:002,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,10,EC,NaN,EC,...,8505213,Bellinzona,01.01.2018 20:10,01.01.2018 20:08:39,GESCHAETZT,01.01.2018 20:13,01.01.2018 20:14:36,GESCHAETZT,False,False
317,01.01.2018,85:11:10:002,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,10,EC,NaN,EC,...,8505004,Arth-Goldau,01.01.2018 21:09,01.01.2018 21:08:24,GESCHAETZT,01.01.2018 21:13,01.01.2018 21:14:10,GESCHAETZT,False,False


### 2. Train's departure is delayed (abfahrtsverspatung)

In [18]:
# Train's departure is delayed
df.loc[(df['ABFAHRTSZEIT'].notnull()) & (df['AB_PROGNOSE'].notnull()), 'abfahrtsverspatung'] = (df['ABFAHRTSZEIT'] < df['AB_PROGNOSE'])
df.head()

,BETRIEBSTAG,FAHRT_BEZEICHNER,BETREIBER_ID,BETREIBER_ABK,BETREIBER_NAME,PRODUKT_ID,LINIEN_ID,LINIEN_TEXT,UMLAUF_ID,VERKEHRSMITTEL_TEXT,...,HALTESTELLEN_NAME,ANKUNFTSZEIT,AN_PROGNOSE,AN_PROGNOSE_STATUS,ABFAHRTSZEIT,AB_PROGNOSE,AB_PROGNOSE_STATUS,DURCHFAHRT_TF,ankunftsverspatung,abfahrtsverspatung
212,01.01.2018,80:L7____:87875:000,80:L7____,SBB,SBB GmbH,Zug,87875,S6,NaN,S,...,Basel Bad Bf,01.01.2018 01:14,NaN,UNBEKANNT,NaN,NaN,PROGNOSE,False,NaN,NaN
314,01.01.2018,85:11:10:002,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,10,EC,NaN,EC,...,Chiasso,01.01.2018 19:10,01.01.2018 19:11:15,PROGNOSE,01.01.2018 19:15,01.01.2018 19:17:01,PROGNOSE,False,True,True
315,01.01.2018,85:11:10:002,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,10,EC,NaN,EC,...,Lugano,01.01.2018 19:40,01.01.2018 19:38:14,GESCHAETZT,01.01.2018 19:42,01.01.2018 19:43:14,GESCHAETZT,False,False,True
316,01.01.2018,85:11:10:002,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,10,EC,NaN,EC,...,Bellinzona,01.01.2018 20:10,01.01.2018 20:08:39,GESCHAETZT,01.01.2018 20:13,01.01.2018 20:14:36,GESCHAETZT,False,False,True
317,01.01.2018,85:11:10:002,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,10,EC,NaN,EC,...,Arth-Goldau,01.01.2018 21:09,01.01.2018 21:08:24,GESCHAETZT,01.01.2018 21:13,01.01.2018 21:14:10,GESCHAETZT,False,False,True


### 3. Get geoposition of stations 

In [29]:
# Load all data about stations where SBB is active
filepath = 'dienststellen-gemass-opentransportdataswiss.csv'
df2 = pd.read_csv(filepath, delimiter=';')
df2.columns

Index(['DIDOK NUMMER', 'BPUIC', 'BEZEICHNUNG_OFFIZIELL', 'Abkuerzung',
       'ORTSCHAFTSNAME', 'GEMEINDENAME', 'BFS_NUMMER', 'BEZIRKSNAME',
       'BEZIRKSNUM', 'KANTONSNAME', 'KANTONSKUERZEL', 'KANTONSNUM',
       'LAND_ISO2_GEO', 'BP_ART_BEZEICHNUNG_DE', 'BP_BETRIEBSPUNKT_ART_ID',
       'BPOF_ART_BEZEICHNUNG_DE', 'BPOF_BETRIEBSPUNKT_ART_ID',
       'BPTF_ART_BEZEICHNUNG_DE', 'BPVB_ART_BEZEICHNUNG_DE',
       'BPVB_BETRIEBSPUNKT_ART_ID', 'BPVH_BEZEICHNUNG_ALT', 'BPVH_EPR_CODE',
       'BPVH_IATA_CODE', 'E_LV95', 'N_LV95', 'Z_LV95', 'E_LV03', 'N_LV03',
       'Z_LV03', 'E_WGS84', 'N_WGS84', 'Z_WGS84', 'BPVH_VERKEHRSMITTEL',
       'BPVH_VERKEHRSMITTEL_TEXT_DE', 'GO_IDENTIFIKATION', 'tu-nummer',
       'GO_ABKUERZUNG_DE', 'GO_BEZEICHNUNG_DE', 'dst_abk', 'lod', 'geopos'],
      dtype='object')

In [30]:
# Extract geopos of stations in a new dataframe geo_pos
df2['BPVH_VERKEHRSMITTEL_TEXT_DE'].fillna('zug', inplace=True)
df2 = df2.drop(df2[(df2['BPVH_VERKEHRSMITTEL_TEXT_DE'].str.lower().str.find('zug') == -1)].index)
geo_pos = df2[['BEZEICHNUNG_OFFIZIELL', 'geopos']].drop_duplicates()
geo_pos.head()

,BEZEICHNUNG_OFFIZIELL,geopos
25,La Plaine-Frontière,"46.1774699611,5.99058007065"
64,Zimeysa,"46.221239603,6.06598529651"
99,Châtelaine (bif),"46.2171399788,6.10257999807"
122,Furet (bif),"46.2087699868,6.11549993836"
136,Tunnel du Saut-de-Mouton,"46.2073036331,6.11975359846"


In [31]:
# Check if all stations in df are present in geo_pos
exists = df['HALTESTELLEN_NAME'].drop_duplicates().isin(geo_pos['BEZEICHNUNG_OFFIZIELL'])
exists.value_counts()

True     593
False      8
Name: HALTESTELLEN_NAME, dtype: int64

In [28]:
# Let's take a look at these stations since their number is low
not_present_stations = df[(np.logical_not(df['HALTESTELLEN_NAME'].isin(geo_pos['BEZEICHNUNG_OFFIZIELL'])))].HALTESTELLEN_NAME.drop_duplicates()
not_present_stations

3575                               Puidoux-Chexbres
4522                                Ecublens-Rue FR
6826                                            Wil
30005                                     Oberkirch
35699                                        Benken
38816                                  Ambri-Piotta
49574    Biel/Bienne Bözingenfeld/Champs-de-Boujean
52595                                        Wohlen
Name: HALTESTELLEN_NAME, dtype: object

Looking up these stations we notice that the names sometimes contain an abbreviation of the canton (Oberkirch LU -> Oberkirch), special characters (Ambrì-Piotta -> Ambri-Piotta) or some of the stations have not been marked as train ('Zug') stations. We manually update the dataframe.

In [32]:
# Manually update the dataframe
geo_pos.loc[geo_pos['BEZEICHNUNG_OFFIZIELL'] == 'Puidoux', 'BEZEICHNUNG_OFFIZIELL'] = 'Puidoux-Chexbres'
geo_pos.loc[geo_pos['BEZEICHNUNG_OFFIZIELL'] == 'Ecublens-Rue', 'BEZEICHNUNG_OFFIZIELL'] = 'Ecublens-Rue FR'
geo_pos.loc[geo_pos['BEZEICHNUNG_OFFIZIELL'] == 'Wil SG', 'BEZEICHNUNG_OFFIZIELL'] = 'Wil'
geo_pos.loc[geo_pos['BEZEICHNUNG_OFFIZIELL'] == 'Oberkirch LU', 'BEZEICHNUNG_OFFIZIELL'] = 'Oberkirch'
geo_pos.loc[geo_pos['BEZEICHNUNG_OFFIZIELL'] == 'Benken SG', 'BEZEICHNUNG_OFFIZIELL'] = 'Benken'
geo_pos.loc[geo_pos['BEZEICHNUNG_OFFIZIELL'] == 'Ambrì-Piotta', 'BEZEICHNUNG_OFFIZIELL'] = 'Ambri-Piotta'
geo_pos.loc[geo_pos['BEZEICHNUNG_OFFIZIELL'] == 'Biel/Bienne Bözingenfeld/Champ', 'BEZEICHNUNG_OFFIZIELL'] = 'Biel/Bienne Bözingenfeld/Champs-de-Boujean'
geo_pos.loc[geo_pos['BEZEICHNUNG_OFFIZIELL'] == 'Wohlen AG', 'BEZEICHNUNG_OFFIZIELL'] = 'Wohlen'
geo_pos.loc[geo_pos['BEZEICHNUNG_OFFIZIELL'] == 'Langnau i.E.', 'BEZEICHNUNG_OFFIZIELL'] = 'Langnau'
geo_pos.loc[geo_pos['BEZEICHNUNG_OFFIZIELL'] == 'Chamoson-St-Pierre-de-Clages', 'BEZEICHNUNG_OFFIZIELL'] = 'Chamoson'
geo_pos.loc[geo_pos['BEZEICHNUNG_OFFIZIELL'] == 'Riedbach', 'BEZEICHNUNG_OFFIZIELL'] = 'Riedbach BE'
geo_pos.loc[geo_pos['BEZEICHNUNG_OFFIZIELL'] == 'Zürich Herdern Abstellgruppe', 'BEZEICHNUNG_OFFIZIELL'] = 'Zürich Herdern'
geo_pos.loc[geo_pos['BEZEICHNUNG_OFFIZIELL'] == 'La Chaux-de-Fonds [voie 1]', 'BEZEICHNUNG_OFFIZIELL'] = 'La Chaux-de-Fonds-Ouest'
geo_pos = geo_pos.append({'BEZEICHNUNG_OFFIZIELL': 'Bressonnaz', 'geopos': '46.6531499774,6.79126002142'}, ignore_index=True)
geo_pos = geo_pos.append({'BEZEICHNUNG_OFFIZIELL': 'Onnens', 'geopos': '46.8325199807,6.68992999974'}, ignore_index=True)
geo_pos = geo_pos.append({'BEZEICHNUNG_OFFIZIELL': 'Ecublens-Rue', 'geopos': '46.6104200378,6.81104998287'}, ignore_index=True)
geo_pos = geo_pos.append({'BEZEICHNUNG_OFFIZIELL': 'Wohlen AG', 'geopos': '47.3484599962,8.26977999385'}, ignore_index=True)
geo_pos = geo_pos.append({'BEZEICHNUNG_OFFIZIELL': 'Oberkirch LU', 'geopos': '47.1552300076,8.11421996366'}, ignore_index=True)
geo_pos = geo_pos.append({'BEZEICHNUNG_OFFIZIELL': 'Benken SG', 'geopos': '47.2044500206,9.00862995731'}, ignore_index=True)
geo_pos = geo_pos.append({'BEZEICHNUNG_OFFIZIELL': 'Ambrì-Piotta', 'geopos': '46.5106100279,8.69015001593'}, ignore_index=True)
geo_pos = geo_pos.append({'BEZEICHNUNG_OFFIZIELL': 'Puidoux', 'geopos': '46.493840003,6.76568002614'}, ignore_index=True)
geo_pos = geo_pos.append({'BEZEICHNUNG_OFFIZIELL': 'Riedbach', 'geopos': '46.9411499578,7.33405003896'}, ignore_index=True)
geo_pos = geo_pos.append({'BEZEICHNUNG_OFFIZIELL': 'Chamoson-St-Pierre-de-Clages', 'geopos': '46.1888399991,7.241090006'}, ignore_index=True)
geo_pos = geo_pos.append({'BEZEICHNUNG_OFFIZIELL': 'Oberwinterthur Unterhaltsanlage', 'geopos': '47.5148000058,8.7678000185'}, ignore_index=True)

In [33]:
# Save geo positions
geo_pos.to_csv('data/geo_pos.csv',sep=';',index=False)

In [35]:
# Double-check
exists = df['HALTESTELLEN_NAME'].drop_duplicates().isin(geo_pos['BEZEICHNUNG_OFFIZIELL'])
exists.value_counts()

True    601
Name: HALTESTELLEN_NAME, dtype: int64

In [38]:
# Double-check
not_present_stations = df[(np.logical_not(df['HALTESTELLEN_NAME'].isin(geo_pos['BEZEICHNUNG_OFFIZIELL'])))].HALTESTELLEN_NAME.drop_duplicates()
not_present_stations

Series([], Name: HALTESTELLEN_NAME, dtype: object)

In [40]:
# Add the geopos in the dataframe
dict_geo_pos = geo_pos.set_index('BEZEICHNUNG_OFFIZIELL').to_dict()['geopos']
df['geopos'] = df['HALTESTELLEN_NAME'].map(dict_geo_pos)
df.head()

,BETRIEBSTAG,FAHRT_BEZEICHNER,BETREIBER_ID,BETREIBER_ABK,BETREIBER_NAME,PRODUKT_ID,LINIEN_ID,LINIEN_TEXT,UMLAUF_ID,VERKEHRSMITTEL_TEXT,...,ANKUNFTSZEIT,AN_PROGNOSE,AN_PROGNOSE_STATUS,ABFAHRTSZEIT,AB_PROGNOSE,AB_PROGNOSE_STATUS,DURCHFAHRT_TF,ankunftsverspatung,abfahrtsverspatung,geopos
212,01.01.2018,80:L7____:87875:000,80:L7____,SBB,SBB GmbH,Zug,87875,S6,NaN,S,...,01.01.2018 01:14,NaN,UNBEKANNT,NaN,NaN,PROGNOSE,False,NaN,NaN,"47.5681499767,7.60730001377"
314,01.01.2018,85:11:10:002,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,10,EC,NaN,EC,...,01.01.2018 19:10,01.01.2018 19:11:15,PROGNOSE,01.01.2018 19:15,01.01.2018 19:17:01,PROGNOSE,False,True,True,"45.8321799683,9.03144999936"
315,01.01.2018,85:11:10:002,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,10,EC,NaN,EC,...,01.01.2018 19:40,01.01.2018 19:38:14,GESCHAETZT,01.01.2018 19:42,01.01.2018 19:43:14,GESCHAETZT,False,False,True,"46.0055000266,8.94687993261"
316,01.01.2018,85:11:10:002,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,10,EC,NaN,EC,...,01.01.2018 20:10,01.01.2018 20:08:39,GESCHAETZT,01.01.2018 20:13,01.01.2018 20:14:36,GESCHAETZT,False,False,True,"46.1954300054,9.02950994767"
317,01.01.2018,85:11:10:002,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,10,EC,NaN,EC,...,01.01.2018 21:09,01.01.2018 21:08:24,GESCHAETZT,01.01.2018 21:13,01.01.2018 21:14:10,GESCHAETZT,False,False,True,"47.0491499738,8.5494899754"
